1. Directorio de Trabajo

In [ ]:
pwd

2. Importar el archivo ==> bbdd es un DATAFRAME

In [ ]:
import pandas as pd

bbdd=pd.read_csv(r'BBDD3.csv',delimiter=';')

print(bbdd)


3. Instalo ultima version de Stats Models

In [ ]:
#conda install -c conda-forge statsmodels3

4. Importo Libreria de Stats Model

In [ ]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from statsmodels.graphics.api import qqplot
from patsy import dmatrices
from statsmodels.tsa.arima.model import ARIMA

5. Analisis de Estacionariedad
https://www.youtube.com/watch?v=warCSvy1DMk

In [ ]:
plt.plot(bbdd['SA'])

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
X=bbdd['SA'].values
results=adfuller(X)
print('ADF Statistics: %f' % results[0])
print('p-value: %f' % results[1])
print('Critical Values:')
for key, value in  results[4].items():
    print ('\t%s: %.3f' % (key,value))

if results[0]<results[4] ["10%"]:
    print("Reject H0 - Time Series Is Stationary")
else:
    print("Failed to Reject H0 - Time Series is Non-Stationary")

6. Definición del orden de los procesos ARMA(p,q)

In [ ]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#AutoML
#!pip install pmdarima


In [ ]:
#a.AutoML
stepwise_fit=auto_arima(bbdd['SA'], trace= True, supresswarnings= True)
stepwise_fit.summary()
#Arima(0,0,0)

In [ ]:
#b.Vamos a analizar ACF & PACF
sm.graphics.tsa.plot_acf(bbdd['SA'].values.squeeze(), lags=10)  #MA
sm.graphics.tsa.plot_pacf(bbdd['SA'].values.squeeze(), lags=10) #AR
plt.show()

7. Arimax(p,q,diferencia)

In [ ]:
#Me fijo las funciones de la libreria
#dir(sm.tsa)

#Defino las Bases
dta = bbdd[['SA', 'Dev t', 'Dev t-1', 'Dev t-2', 'Dev t-3', 'Dev t-4', 'Dev t-5' , 'Dev t-6', 'Dev t-7', 'Dev t-8', 'Dev t-9', 'Dev t-10', 'Dev t-11', 'Dev t-12']]
endogena = dta['SA']
exogena = dta[['Dev t', 'Dev t-1', 'Dev t-2', 'Dev t-3', 'Dev t-4', 'Dev t-5' , 'Dev t-6', 'Dev t-7', 'Dev t-8', 'Dev t-9', 'Dev t-10', 'Dev t-11', 'Dev t-12']]
print(exogena)
exogena_array=exogena.to_numpy()
print(exogena_array)



In [ ]:
#Corro el Modelo
arma_mod20 = ARIMA(endogena, exog=exogena ,order=(0, 0, 0)).fit() #order(p,i,q)
print(arma_mod20.summary())
#Interpretacion de salida de la regresion https://www.youtube.com/watch?v=VvlqA-iO2HA

In [ ]:
#En el grafico de correlacion de Inflacion de Suma y Dev t-k, aparecen como relevantes t-1 & t-6
#En la regresion aparecen significativos: t-1, t-6, t-7 probamos ajustar con esos periodos
endogena2 = dta['SA']
exogena2 = dta[['Dev t', 'Dev t-1', 'Dev t-3', 'Dev t-8', 'Dev t-11']]
print(exogena2)
exogena2_array=exogena2.to_numpy()
print(exogena2_array)

In [ ]:
#Corro el Modelo Iteracion 2
arma_mod20_2 = ARIMA(endogena2, exog=exogena2 ,order=(0, 0, 0)).fit()
print(arma_mod20_2.summary())


Tenemos la ecuación de Inflación de Autopartes: Inf_Repuestos(t)=0.0115+0.37*Dev(t)+0.103*Dev(t-1)+0.0741*Dev(t-9)

In [ ]:
#Aplicamos el modelo para ver errores

residuals = pd.DataFrame(arma_mod20_2.resid)
residuals.plot()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())